In [3]:
install.packages("leaps")
install.packages("Metrics")

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



In [84]:
library(tidyverse)
library(lubridate)

library(ggplot2)
library(broom)
library(tidyr)
library(dplyr)
library(Metrics)
library(modelr)

library(leaps)
library(caret)

- In the previous assignment, I explored the data with useful visuals that allowed me to observe relationships between input variables and response one and among input variables themselves. I also observed how some input variables were highly correlated (multicollinearity) by plotting a heatmap.
- in this assignment, I will first convert the 'Created_At' and 'Updated_AT' Date variables to numerical values, where they represent the days since created, updated respectively.
- Then, I will clean the Data and use the Forward Selection Algorithm to find the best-fit model.
- Afterward, I am going to split the data into training, testing data with a ratio of 80/20.
- then, I am going to use the variables from the Forward Selection results to fit an Additive linear regression model.
- Then, using the predict function, I am going to predict the number of stars in the testing set using the fitted model.
- Finally, I will calculate the RMSE to evaluate the accuracy of the prediction model.*


In [141]:
df <- read_csv('repositories.csv')

Rows: 215029 Columns: 24
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (8): Name, Description, URL, Homepage, Language, License, Topics, Defau...
dbl  (5): Size, Stars, Forks, Issues, Watchers
lgl  (9): Has Issues, Has Projects, Has Downloads, Has Wiki, Has Pages, Has ...
dttm (2): Created At, Updated At

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [142]:
df_subset <- df %>% sample_n(size = 10000)

head(df_subset, n = 3)

Name,Description,URL,Created At,Updated At,Homepage,Size,Stars,Forks,Issues,⋯,Has Issues,Has Projects,Has Downloads,Has Wiki,Has Pages,Has Discussions,Is Fork,Is Archived,Is Template,Default Branch
<chr>,<chr>,<chr>,<dttm>,<dttm>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<chr>
monoio,Rust async runtime based on io-uring.,https://github.com/bytedance/monoio,2021-11-30 08:25:46,2023-09-21 08:50:14,NA,5178,3020,171,24,⋯,TRUE,TRUE,TRUE,TRUE,FALSE,TRUE,FALSE,FALSE,FALSE,master
laychat,layIM+workerman+thinkphp5的webIM即时通讯系统 V2.0,https://github.com/nick-bai/laychat,2016-08-30 06:32:30,2023-09-05 07:58:21,NA,10294,431,232,6,⋯,TRUE,TRUE,TRUE,TRUE,FALSE,FALSE,FALSE,FALSE,FALSE,master
raw-wasm,Raw WebAssembly demos,https://github.com/binji/raw-wasm,2019-05-30 05:56:27,2023-09-15 08:47:52,https://binji.github.io/raw-wasm,582,393,16,0,⋯,TRUE,FALSE,TRUE,FALSE,TRUE,FALSE,FALSE,FALSE,FALSE,main


*I am choosing only those variables because of the limitation on running the Forward Selection Algorithm on all of the given this variables. Also, the other columns (e.g. logical variables) contain very redundant data.  
Note: I explain more about this issue in the Forward Selection cell down below.*

In [ ]:
features <- c("Size", "Stars", "Forks",
              "Issues", 'Watchers', 'Is Archived', 'Has Issues', 'Has Pages',
              'Created At', 'Updated At')
df_subset_1 <- df_subset %>% select(all_of(features))

head(df_subset_1, n = 3)
nrow(df_subset_1)
sum(is.na(df_subset_1))

Size,Stars,Forks,Issues,Watchers,Is Archived,Has Issues,Has Pages,Created At,Updated At
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>,<lgl>,<lgl>,<dttm>,<dttm>
5178,3020,171,24,3020,FALSE,TRUE,FALSE,2021-11-30 08:25:46,2023-09-21 08:50:14
10294,431,232,6,431,FALSE,TRUE,FALSE,2016-08-30 06:32:30,2023-09-05 07:58:21
582,393,16,0,393,FALSE,TRUE,TRUE,2019-05-30 05:56:27,2023-09-15 08:47:52


[1] 10000

[1] 0

In [149]:
df_subset$'Created At' <- as.Date(strptime(df_subset$'Created At', format = "%Y-%m-%d %H:%M:%S"))
df_subset$'Updated At' <- as.Date(strptime(df_subset$'Updated At', format = "%Y-%m-%d %H:%M:%S"))

In [150]:
df_subset_2 <- df_subset_1 %>%
  mutate(
    `Created At` = as.Date(`Created At`),        
    `Updated At` = as.Date(`Updated At`),        
    repo_age_days = as.numeric(Sys.Date() - `Created At`),
    days_since_update = as.numeric(Sys.Date() - `Updated At`)
  )

head(df_subset_2, n = 3)

Size,Stars,Forks,Issues,Watchers,Is Archived,Has Issues,Has Pages,Created At,Updated At,repo_age_days,days_since_update
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>,<lgl>,<lgl>,<date>,<date>,<dbl>,<dbl>
5178,3020,171,24,3020,FALSE,TRUE,FALSE,2021-11-30,2023-09-21,1094,434
10294,431,232,6,431,FALSE,TRUE,FALSE,2016-08-30,2023-09-05,3012,450
582,393,16,0,393,FALSE,TRUE,TRUE,2019-05-30,2023-09-15,2009,440


In [151]:
df_subset_3 <- df_subset_2 %>% select(-c('Created At', 'Updated At'))

head(df_subset_3, n = 3)

Size,Stars,Forks,Issues,Watchers,Is Archived,Has Issues,Has Pages,repo_age_days,days_since_update
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>,<lgl>,<lgl>,<dbl>,<dbl>
5178,3020,171,24,3020,FALSE,TRUE,FALSE,1094,434
10294,431,232,6,431,FALSE,TRUE,FALSE,3012,450
582,393,16,0,393,FALSE,TRUE,TRUE,2009,440


In [152]:
non_na_repo_age <- df_subset_3 %>% filter(!is.na(repo_age_days))
non_na_days_since_update <- df_subset_3 %>% filter(!is.na(days_since_update))

head(df_subset_3 %>% select(repo_age_days, days_since_update, n = 3))

repo_age_days,days_since_update,n
<dbl>,<dbl>,<dbl>
1094,434,171
3012,450,232
2009,440,16
2841,432,77
2078,443,50
4091,440,117


In [153]:
df_cleaned <- df_subset_3 %>% filter(!is.na(Stars) & !is.na(Forks) & !is.na(repo_age_days))  

head(df_cleaned, n = 3)

Size,Stars,Forks,Issues,Watchers,Is Archived,Has Issues,Has Pages,repo_age_days,days_since_update
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>,<lgl>,<lgl>,<dbl>,<dbl>
5178,3020,171,24,3020,FALSE,TRUE,FALSE,1094,434
10294,431,232,6,431,FALSE,TRUE,FALSE,3012,450
582,393,16,0,393,FALSE,TRUE,TRUE,2009,440


In [154]:
summary_stats <- df_cleaned %>%
  summarize(
    count = n(),
    avg_stars = mean(Stars, na.rm = TRUE),
    avg_forks = mean(Forks, na.rm = TRUE),
    avg_issues = mean(Issues, na.rm = TRUE),
    avg_repo_age = mean(repo_age_days, na.rm = TRUE),
    avg_days_since_update = mean(days_since_update, na.rm = TRUE)
  )

print(summary_stats)
head(df_cleaned, n = 3)
nrow(df_cleaned)
sum(is.na(df_cleaned))

# A tibble: 1 × 6
  count avg_stars avg_forks avg_issues avg_repo_age avg_days_since_update
  <int>     <dbl>     <dbl>      <dbl>        <dbl>                 <dbl>
1 10000     1088.      230.       38.5        2747.                  453.


Size,Stars,Forks,Issues,Watchers,Is Archived,Has Issues,Has Pages,repo_age_days,days_since_update
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>,<lgl>,<lgl>,<dbl>,<dbl>
5178,3020,171,24,3020,FALSE,TRUE,FALSE,1094,434
10294,431,232,6,431,FALSE,TRUE,FALSE,3012,450
582,393,16,0,393,FALSE,TRUE,TRUE,2009,440


[1] 10000

[1] 0

*the limitation I faced with the variables I deleted is that they had many missing values. therefore, I decided to keep the same variables I used in previous assignment.*

In [155]:
set.seed(123) # DO NOT CHANGE!

GitHub_sample <-
    df_cleaned %>%
    mutate(id = row_number())

training_GitHub <- 
    GitHub_sample %>%
    slice_sample(prop = 0.80, replace = FALSE)

testing_GitHub <- 
    GitHub_sample %>% 
    anti_join(training_GitHub, by = "id") %>%
    select(-id)

training_GitHub <- 
    training_GitHub %>% 
    select(-id)

head(training_GitHub)
nrow(training_GitHub)

nrow(testing_GitHub)

Size,Stars,Forks,Issues,Watchers,Is Archived,Has Issues,Has Pages,repo_age_days,days_since_update
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>,<lgl>,<lgl>,<dbl>,<dbl>
5085,710,126,30,710,FALSE,TRUE,FALSE,1811,430
30,388,36,19,388,FALSE,TRUE,FALSE,1613,435
9,468,20,0,468,FALSE,TRUE,FALSE,1562,438
59,728,173,38,728,FALSE,TRUE,FALSE,3875,446
9250,277,152,82,277,FALSE,TRUE,FALSE,4781,532
506,338,27,22,338,FALSE,TRUE,FALSE,4256,437


[1] 8000

[1] 2000

- One of the limitations I faced when runing the Forward Selection Algorithm on the given data is that The kernel will collapse and the restart entirly.
- To solve this problem, I will drop some of the catogrical variables that have many levels (>2) and I, from the experince I gained in this course belive they are in little to no utillity in predicting the number of stars.

In [156]:
GitHub_forward_sel <- regsubsets(
  Stars ~ ., data = training_GitHub,
  method = "forward",
  nvmax = ncol(training_GitHub) - 1
)
GitHub_forward_sel

GitHub_fwd_summary <- summary(GitHub_forward_sel)

GitHub_fwd_summary <- tibble(
  n_input_variables = 1:length(GitHub_fwd_summary$rss),  
  RSS = GitHub_fwd_summary$rss,
  BIC = GitHub_fwd_summary$bic,
  Cp = GitHub_fwd_summary$cp
)

Subset selection object
Call: regsubsets.formula(Stars ~ ., data = training_GitHub, method = "forward", 
    nvmax = ncol(training_GitHub) - 1)
9 Variables  (and intercept)
                  Forced in Forced out
Size                  FALSE      FALSE
Forks                 FALSE      FALSE
Issues                FALSE      FALSE
Watchers              FALSE      FALSE
`Is Archived`TRUE     FALSE      FALSE
`Has Issues`TRUE      FALSE      FALSE
`Has Pages`TRUE       FALSE      FALSE
repo_age_days         FALSE      FALSE
days_since_update     FALSE      FALSE
1 subsets of each size up to 9
Selection Algorithm: forward

In [157]:
GitHub_fwd_summary
summary(GitHub_forward_sel)

n_input_variables,RSS,BIC,Cp
<int>,<dbl>,<dbl>,<dbl>
1,2.029885e-19,-547191.3,8625.483406
2,1.044763e-19,-552495.8,560.923043
3,9.801435e-20,-552997.6,33.793967
4,9.759576e-20,-553022.9,1.518474
5,9.758537e-20,-553014.7,2.667678
6,9.758158e-20,-553006.1,4.357361
7,9.757844e-20,-552997.3,6.100333
8,9.757758e-20,-552988.4,8.029938
9,9.757721e-20,-552979.4,10.000000


Subset selection object
Call: regsubsets.formula(Stars ~ ., data = training_GitHub, method = "forward", 
    nvmax = ncol(training_GitHub) - 1)
9 Variables  (and intercept)
                  Forced in Forced out
Size                  FALSE      FALSE
Forks                 FALSE      FALSE
Issues                FALSE      FALSE
Watchers              FALSE      FALSE
`Is Archived`TRUE     FALSE      FALSE
`Has Issues`TRUE      FALSE      FALSE
`Has Pages`TRUE       FALSE      FALSE
repo_age_days         FALSE      FALSE
days_since_update     FALSE      FALSE
1 subsets of each size up to 9
Selection Algorithm: forward
         Size Forks Issues Watchers `Is Archived`TRUE `Has Issues`TRUE
1  ( 1 ) " "  " "   " "    "*"      " "               " "             
2  ( 1 ) " "  "*"   " "    "*"      " "               " "             
3  ( 1 ) " "  "*"   "*"    "*"      " "               " "             
4  ( 1 ) "*"  "*"   "*"    "*"      " "               " "             
5  ( 1 ) "*"  "*"   "*

*based on the forward selection, the variables for the best-fit model are Size Forks Issues Watchers. Therefore I will fit the training data into a linear regression prediction model using an additive model for all the variables we just selected(4). the reason I selected model (4) is that it has the best combination of lowest CP, BIC and the among the highest RSS.*

In [161]:
GitHub_full <- lm(Stars ~ Size + Forks + Issues + Watchers, data = training_GitHub)

GitHub_full


Call:
lm(formula = Stars ~ Size + Forks + Issues + Watchers, data = training_GitHub)

Coefficients:
(Intercept)         Size        Forks       Issues     Watchers  
 -6.663e-13   -1.241e-19   -4.173e-16   -1.169e-15    1.000e+00  


In [162]:
GitHub_test_pred_full <- predict(GitHub_full, newdata = testing_GitHub)

head(GitHub_test_pred_full)

1   2   3   4   5   6 
261 239 720 289 615 471

In [163]:
GitHub_RMSE_model <- tibble(
    Model = "Full Regression",
    RMSE = rmse(model = GitHub_full,
                data = testing_GitHub))

GitHub_RMSE_model 

Model,RMSE
<chr>,<dbl>
Full Regression,1.286056e-12


*By using the vif function, and to resolve multicollinearity, I got the following results: Size: 1.001, Forks: 1.615, Issues: 1.441, Watchers: 2.042, `Has Discussions`: 1.051repo_age_days: 1.087, days_since_update: 1.071.
and I saw that 'Watchers' has a strong correlation with both 'Forks' and 'Issues' (0.62 and 0.55 respectively). Now lets fit a linear regression model without 'Watchers'
Note: I could have removed 'Watchers' before running the Forward Selection Algorithm, however, I wanted to see what will the algorithm select as the best-fit model.*

In [164]:
GitHub_No_Watchers <- lm(Stars ~ Size + Forks + Issues + `Is Archived`, data = training_GitHub)

GitHub_No_Watchers


Call:
lm(formula = Stars ~ Size + Forks + Issues + `Is Archived`, data = training_GitHub)

Coefficients:
      (Intercept)               Size              Forks             Issues  
        9.096e+02          1.667e-04          6.151e-01          1.642e+00  
`Is Archived`TRUE  
       -2.525e+02  


In [165]:
GitHub_pred_No_Watchers <- predict(GitHub_No_Watchers, newdata = testing_GitHub)

head(GitHub_pred_No_Watchers)

1         2         3         4         5         6 
 981.6223  930.2573 1000.2979 1015.7820 1045.4375  988.7518

In [166]:
GitHub_RMSE_No_Watchers <- tibble(
    Model = "Full Regression No Watchers",
    RMSE = rmse(model = GitHub_No_Watchers,
                data = testing_GitHub))

GitHub_RMSE_No_Watchers

Model,RMSE
<chr>,<dbl>
Full Regression No Watchers,1934.575


*Since we saw in the previous assignment visualization that 'Watchers' was perfectly linearly correlated with Stars.  Now, let's fit a simple linear regression model with only 'Forks' as an input.*

In [184]:
GitHub_Simple_model <- lm(Stars ~ Forks, data = training_GitHub)

GitHub_Simple_model


Call:
lm(formula = Stars ~ Forks, data = training_GitHub)

Coefficients:
(Intercept)        Forks  
   950.6983       0.6777  


In [185]:
GitHub_test_pred_Simple <- predict(GitHub_full, newdata = testing_GitHub)

head(GitHub_test_pred_full)

1   2   3   4   5   6 
261 239 720 289 615 471

In [186]:
GitHub_RMSE_Simple_models <- tibble(
    Model = "OLS Simple Regression",
    RMSE = rmse(model = GitHub_Simple_model,
                data = testing_GitHub))

GitHub_RMSE_Simple_models

Model,RMSE
<chr>,<dbl>
OLS Simple Regression,1998.3
